In [55]:
# Data manipulation
import pandas as pd
import numpy as np
# Evaluation of the model
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.size'] = 18
%matplotlib inline

MAX_EVALS = 5
import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer



DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [56]:
import json
import shlex
import subprocess
import logging
from pprint import pprint
import re
import os


In [72]:
def runthebenchmark(hyperparameters):
    os.chdir('/home/dsinghvi/project/progress/active/../')
    out=subprocess.Popen(["python3","read_config.py"], shell=False, stdout=subprocess.PIPE)
    logging.basicConfig(level=logging.DEBUG)
    output=out.stdout.read().decode('utf-8')
    value = output.split(" ")[5]
    value = float(value)
    print(value)
    return float(1/value)
    #logging.basicConfig(filename='app.log', filemode='a', format='%(name)s - %(levelname)s - %(message)s',level=logging.DEBUG)


In [93]:
def objective(hyperparameters):
    global ITERATION
    ITERATION += 1
    start = timer()
    result = runthebenchmark(hyperparameters)
    run_time = timer() - start
    print({'result': result, 'hyperparameters': hyperparameters, 'iteration': ITERATION, 'iteration_time': run_time, 'status': STATUS_OK})
    return {'loss': float(result), 'hyperparameters': hyperparameters, 'iteration': ITERATION, 'iteration_time': run_time, 'status': STATUS_OK}
    

In [94]:
from hyperopt import hp
from hyperopt.pyll.stochastic import sample

space = {
    'romio_ds_read' : hp.choice('romio_ds_read',[{'enable':'enable'},{'disable':'disable'}]),
    'romio_ds_write' : hp.choice('romio_ds_write',[{'enable':'enable'},{'disable':'disable'}]),
    'setstripe' : hp.qloguniform('setstripe',0,20,1),
    'count' : hp.quniform('count',0,20,1)
}

In [95]:
x = sample(space)
print(x)

{'count': 8.0, 'romio_ds_read': {'disable': 'disable'}, 'romio_ds_write': {'enable': 'enable'}, 'setstripe': 71.0}


In [96]:
from hyperopt import tpe

# Create the algorithm
tpe_algorithm = tpe.suggest


from hyperopt import Trials

# Record results
trials = Trials()

from hyperopt import fmin


ITERATION = 0

best = fmin(fn = objective, space = space, algo = tpe.suggest, trials = trials, max_evals = MAX_EVALS)


  0%|          | 0/5 [00:00<?, ?it/s, best loss: ?]

INFO:hyperopt.tpe:tpe_transform took 0.006050 seconds
INFO:hyperopt.tpe:TPE using 0 trials


357.47                                             
{'result': 0.0027974375472067585, 'hyperparameters': {'count': 0.0, 'romio_ds_read': {'disable': 'disable'}, 'romio_ds_write': {'disable': 'disable'}, 'setstripe': 431578862.0}, 'iteration': 1, 'iteration_time': 6.1296276999637485, 'status': 'ok'}
 20%|██        | 1/5 [00:06<00:24,  6.15s/it, best loss: 0.0027974375472067585]

INFO:hyperopt.tpe:tpe_transform took 0.006394 seconds
INFO:hyperopt.tpe:TPE using 1/1 trials with best loss 0.002797


355.85                                                                         
{'result': 0.002810172825628776, 'hyperparameters': {'count': 1.0, 'romio_ds_read': {'disable': 'disable'}, 'romio_ds_write': {'enable': 'enable'}, 'setstripe': 126117217.0}, 'iteration': 2, 'iteration_time': 4.395754866302013, 'status': 'ok'}
 40%|████      | 2/5 [00:10<00:16,  5.63s/it, best loss: 0.0027974375472067585]

...

352.8                                                                          
{'result': 0.002834467120181406, 'hyperparameters': {'count': 6.0, 'romio_ds_read': {'disable': 'disable'}, 'romio_ds_write': {'disable': 'disable'}, 'setstripe': 400641.0}, 'iteration': 3, 'iteration_time': 6.06619977299124, 'status': 'ok'}
 60%|██████    | 3/5 [00:16<00:11,  5.77s/it, best loss: 0.0027974375472067585]

INFO:hyperopt.tpe:tpe_transform took 0.006380 seconds
INFO:hyperopt.tpe:TPE using 3/3 trials with best loss 0.002797


368.8                                                                          
{'result': 0.0027114967462039045, 'hyperparameters': {'count': 13.0, 'romio_ds_read': {'enable': 'enable'}, 'romio_ds_write': {'disable': 'disable'}, 'setstripe': 108901.0}, 'iteration': 4, 'iteration_time': 6.055659491568804, 'status': 'ok'}
 80%|████████  | 4/5 [00:22<00:05,  5.86s/it, best loss: 0.0027114967462039045]

INFO:hyperopt.tpe:tpe_transform took 0.006301 seconds
INFO:hyperopt.tpe:TPE using 4/4 trials with best loss 0.002711


332.23                                                                         
{'result': 0.003009962977455377, 'hyperparameters': {'count': 16.0, 'romio_ds_read': {'disable': 'disable'}, 'romio_ds_write': {'enable': 'enable'}, 'setstripe': 6.0}, 'iteration': 5, 'iteration_time': 4.0869356812909245, 'status': 'ok'}
100%|██████████| 5/5 [00:26<00:00,  5.33s/it, best loss: 0.0027114967462039045]


In [ ]:
###Loader
data = pd.read_csv('../stats.txt', delim_whitespace=True)
array = data.values

output_col = input("Number of output column indexed from 0")
output_col = 2#int(output_col)
(n,m) = array.shape
print(array)                                                                           
X = array[:,0:m]
Y = array[:,output_col]
Y=Y.astype('int')

X=np.delete(X,output_col,1)

k=0
for i in range(0,m-1):
    if type(X[0][k]) == str:
        X=np.delete(X,k,1)
    #    print(X)
    else : 
        k=k+1
print(X)
###Model
model1(X,Y)


